# Loop tiling

---
**Requirements:**

- [Get started](./Get_started.ipynb)
- [Data Management](./Data_management.ipynb)
---

Nested loops often reuse the same data across their iterations and keeping the working set inside the caches can improve performance.
Tiling is a partitioning method of the loops into blocks. It reorders the loops so that each block will repeatedly hit the cache.
A first usage restriction will thus be on the loops' nature itself: not all loops can benefit from tiling, only the ones that will reuse data while showing a poor data locality, thus leading to frequent cache misses.

<img alt="Tiles example" src="../../pictures/tiles.png" style="float:none"/>

OpenACC allows to improve data locality inside loops with the dedicated _tile_ clause.
It specifies the compiler to split each loop in the nest into 2 loops, with an outer set of tile loops and an inner set of element loops.

## Syntax

The tile clause may appear with the _loop_ directive for nested loops.
For _N_ nested loops, the tile clause can take _N_ arguments. The first one being the size of the inner loop of the nest, the last one being the size of the outer loop.

```c
#pragma acc loop tile(32,32)
for(int i = 0 ; i < size_i ; ++i)
{
    for(int j = 0 ; j < size_j ; ++j)
    {
        // A Fabulous calculation
    }
}    
```

## Restrictions

- the tile size (corresponding to the product of the arguments of the tile clause) can be up to 1024
- for better performance the size for the inner loop is a power of 2 (best with 32 to fit a cuda warp)
- if the vector clause is specified, it is then applied to the element loop
- if the gang clause is specified, it is then applied to the tile loop
- the worker clause is applied to the element loop only if the vector clause is not specified

## Example

In the following example, tiling is used to solve a matrix multiplication followed by an addition. Let us take a look at the performance of the naïve algorithm and the manual tiling on CPU.

Example stored in: `../../examples/C/Loop_tiling_example_cpu.c`

In [ ]:
%%idrrun
#include <stdio.h>
#include <stdint.h>
#include <stdlib.h>
#include <time.h>

#define MIN(a,b) ( ((a)<(b))?(a):(b) )

double double_random(){
    return (double) (rand()) / RAND_MAX;
}	

void nullify(int ni, int nj, double* d){
    for (int i=0; i<ni; i++){
       for (int j=0; j<nj; j++){
           d[j+i*nj] = 0.0;
       }
    }
}

double checksum(int ni, int nj, double* d){
    double dsum = 0.0;
    for (int i=0; i<ni; i++){
       for (int j=0; j<nj; j++){
           dsum = dsum + d[j+i*nj];
       }
    }
    return dsum;
}	

void naive_matmul(int ni, int nj, int nk, double* a, double* b, double* c, double* d){
    for (int i=0; i<ni; i++){
       for (int j=0; j<nj; j++){
           for (int k=0; k<nk; k++){
                d[i*nj +j] = d[i*nj +j] + a[k+i*nk] * b[j+k*nj];
           }
           d[j+i*nj]= d[j+i*nj] + c[j+i*nj];
       }
    }
}

void tiled_matmul(int tile, int ni, int nj, int nk, double* a, double* b, double* c, double* d){
     for (int i=0; i<ni; i+=tile){	
       for (int j=0; j<nj; j+=tile){	     
          for (int ii=i; ii< MIN(i+tile,ni); ii++){
            for (int jj=j; jj<MIN(j+tile,nj); jj++){
	         for (int k=0; k<nk; k++){
                       d[ii*nj +jj] = d[ii*nj +jj] + a[k+ii*nk] * b[jj+k*nj];
                 }
             }
          }
       }
    }	
    for (int i=0; i<ni; i++){
       for (int j=0; j<nj; j++){
           d[j+i*nj]= d[j+i*nj] + c[j+i*nj];
       }
    }
	     
}

int main(void)
{
    int ni=4280, nj=4024, nk=1960;
 
    clock_t t1, t2;

    double* a = (double*) malloc(ni*nk*sizeof(double));
    double* b = (double*) malloc(nk*nj*sizeof(double));
    double* c = (double*) malloc(ni*nj*sizeof(double));
    double* d = (double*) malloc(ni*nj*sizeof(double));
    double test;

    unsigned int seed = 1234;
    srand(seed);

    for (int i=0; i<ni; i++){
       for (int k=0; k<nk; k++){   
           a[k+i*nk] = double_random();
       }
    }

    for (int k=0; k<nk; k++){
       for (int j=0; j<nj; j++){   
           b[j+k*nj] = double_random();
       }   
    }  

    for (int i=0; i<ni; i++){
       for (int j=0; j<nj; j++){
           c[j+i*nj] = 2.0;
       }
    }

    nullify(ni, nj, d);
    
    t1 = clock();
    naive_matmul(ni, nj, nk, a, b, c, d);
    t2 = clock();
    test = checksum(ni, nj, d);
    fprintf(stderr, "CPU naive Elapsed: %lf\n", (double) (t2-t1) /CLOCKS_PER_SEC);
    fprintf(stderr, "\tchecksum=%lf\n\n", test);
    nullify(ni, nj, d);

    int tile = 512;
    t1 = clock();
    tiled_matmul(tile, ni, nj, nk, a, b, c, d);
    t2 = clock();
    test = checksum(ni, nj, d);
    fprintf(stderr, "CPU Manually tiled Elapsed: %lf\n", (double) (t2-t1) /CLOCKS_PER_SEC);
    fprintf(stderr, "\tchecsum=%lf\n\n", test);
    nullify(ni, nj, d);

    free(a);
    free(b);
    free(c);
    free(d);

    return 0;
}	

And now it's GPU implementation.

Example stored in: `../../examples/C/Loop_tiling_example_gpu.c`

In [ ]:
%%idrrun -a
#include <stdio.h>
#include <stdint.h>
#include <stdlib.h>
#include <time.h>

#define MIN(a,b) ( ((a)<(b))?(a):(b) )

double double_random(){
    return (double) (rand()) / RAND_MAX;
}	

void nullify(int ni, int nj, double* d){
    for (int i=0; i<ni; i++){
       for (int j=0; j<nj; j++){
           d[j+i*nj] = 0.0;
       }
    }
}

double checksum(int ni, int nj, double* d){
    double dsum = 0.0;
    for (int i=0; i<ni; i++){
       for (int j=0; j<nj; j++){
           dsum = dsum + d[j+i*nj];
       }
    }
    return dsum;
}	

void naive_matmul(int ni, int nj, int nk, double* a, double* b, double* c, double* d){
    #pragma acc parallel loop default(present)
    for (int i=0; i<ni; i++){
       #pragma acc loop	    
       for (int j=0; j<nj; j++){
           for (int k=0; k<nk; k++){
                d[i*nj +j] = d[i*nj +j] + a[k+i*nk] * b[j+k*nj];
           }
           d[j+i*nj]= d[j+i*nj] + c[j+i*nj];
       }
    }
}

void naive_matmul_acc_tiled(int ni, int nj, int nk, double* a, double* b, double* c, double* d){
    #pragma acc parallel loop tile(32,32) default(present)
    for (int i=0; i<ni; i++){
       for (int j=0; j<nj; j++){
           for (int k=0; k<nk; k++){
                d[i*nj +j] = d[i*nj +j] + a[k+i*nk] * b[j+k*nj];
           }
           d[j+i*nj]= d[j+i*nj] + c[j+i*nj];
       }
    }
}


void tiled_matmul(int tile, int ni, int nj, int nk, double* a, double* b, double* c, double* d){
     #pragma acc parallel loop default(present) num_workers(8) vector_length(128)
     #pragma acc loop gang collapse(2)
     for (int i=0; i<ni; i+=tile){	
       for (int j=0; j<nj; j+=tile){	     
          #pragma acc loop worker
          for (int ii=i; ii< MIN(i+tile,ni); ii++){
            #pragma acc loop vector
            for (int jj=j; jj<MIN(j+tile,nj); jj++){
                 #pragma acc loop seq
	         for (int k=0; k<nk; k++){
                       d[ii*nj +jj] = d[ii*nj +jj] + a[k+ii*nk] * b[jj+k*nj];
                 }
             }
          }
       }
    }	
    #pragma acc parallel loop default(present) 
    for (int i=0; i<ni; i++){
       #pragma acc loop
       for (int j=0; j<nj; j++){
           d[j+i*nj]= d[j+i*nj] + c[j+i*nj];
       }
    }
	     
}

int main(void)
{
    int ni=4280, nj=4024, nk=1960;
 
    clock_t t1, t2;

    double* a = (double*) malloc(ni*nk*sizeof(double));
    double* b = (double*) malloc(nk*nj*sizeof(double));
    double* c = (double*) malloc(ni*nj*sizeof(double));
    double* d = (double*) malloc(ni*nj*sizeof(double));
    double test;

    unsigned int seed = 1234;
    srand(seed);

    for (int i=0; i<ni; i++){
       for (int k=0; k<nk; k++){   
           a[k+i*nk] = double_random();
       }
    }

    for (int k=0; k<nk; k++){
       for (int j=0; j<nj; j++){   
           b[j+k*nj] = double_random();
       }   
    }  

    for (int i=0; i<ni; i++){
       for (int j=0; j<nj; j++){
           c[j+i*nj] = 2.0;
       }
    }

    nullify(ni, nj, d);
    
    #pragma acc data copyin(a[0:ni*nk], b[0:nk*nj], c[0:ni*nj]) create(d[0:ni*nj])    
    {
    t1 = clock();
    naive_matmul(ni, nj, nk, a, b, c, d);
    t2 = clock();
    #pragma acc update self(d[0:ni*nj])
    test = checksum(ni, nj, d);
    fprintf(stderr, "GPU naive Elapsed: %lf\n", (double) (t2-t1) /CLOCKS_PER_SEC);
    fprintf(stderr, "\tchecksum=%lf\n\n", test);
    nullify(ni, nj, d);
    #pragma acc update device(d[0:ni*nj])

    t1 = clock();
    naive_matmul_acc_tiled(ni, nj, nk, a, b, c, d);
    t2 = clock();
    #pragma acc update self(d[0:ni*nj])
    test = checksum(ni, nj, d);
    fprintf(stderr, "GPU OpenACC tiled Elapsed: %lf\n", (double) (t2-t1) /CLOCKS_PER_SEC);
    fprintf(stderr, "\tchecksum=%lf\n\n", test);
    nullify(ni, nj, d);
    #pragma acc update device(d[0:ni*nj])    

    int tile = 512;
    t1 = clock();
    tiled_matmul(tile, ni, nj, nk, a, b, c, d);
    t2 = clock();
    #pragma acc update self(d[0:ni*nj])
    test = checksum(ni, nj, d);
    fprintf(stderr, "GPU Manually tiled Elapsed: %lf\n", (double) (t2-t1) /CLOCKS_PER_SEC);
    fprintf(stderr, "\tchecksum=%lf\n\n", test);


    }

    free(a);
    free(b);
    free(c);
    free(d);

    return 0;
}	

## Exercise

In this exercise, you will try to accelerate the numerical resolution of the 2D Laplace's equation with tiles. You can see that tiles parameter should be chosen wisely in order not to deteriorate performance.

Example stored in: `../../examples/C/Loop_tiling_exercise.c`

In [ ]:
%%idrrun -a
#include<stdio.h>
#include<stdlib.h>
#include<math.h>
#include<openacc.h>

int main(void) {
    int    nx = 20000;
    int    ny = 10000;
    int    idx; 
    double T[nx*ny], T_new[nx*ny];
    double erreur;

    for(int i=1; i<nx-1; ++i) {
        for(int j=1; j<ny-1; ++j) {
        T[i*ny+j]     = 0.0;
        T_new[i*ny+j] = 0.0;
        }
    }

    for(int i=0; i<nx; ++i){
        T[i*ny     ] = 100.0;
        T[i*ny+ny-1] = 0.0;
    }

    for(int j=0; j<ny; ++j){
        T[j]           = 0.0;
        T[(nx-1)*ny+j] = 0.0;
    }

    // add acc directive
    for (int it = 0; it<10000; ++it){
        erreur = 0.0;	
        // add acc directive 
        for (int i=1; i<nx-1; ++i) {
            for (int j=1; j<ny-1; ++j) {
                idx = i*(ny)+j;
                T_new[idx] = 0.25*(T[idx+ny]+T[idx-ny] + T[idx+1]+T[idx-1]);
                erreur = fmax(erreur, fabs(T_new[idx]-T[idx]));
            }
         }
        if(it%100 == 0) fprintf(stderr,"it: %d, erreur: %e\n",it,erreur);

        // add acc directive
        for (int i=1; i<nx-1; ++i) {
            for (int j=1; j<ny-1; ++j) {
                T[i*(ny)+j] =  T_new[i*(ny)+j];
            }
        }
    }
    return 0;
}

## Solution

Example stored in: `../../examples/C/Loop_tiling_solution.c`

In [ ]:
%%idrrun -a
#include<stdio.h>
#include<stdlib.h>
#include<math.h>
#include<openacc.h>

int main(void) {
    int    nx = 20000;
    int    ny = 10000; 
    int    idx;    
    double T[nx*ny], T_new[nx*ny];
    double erreur;

    for(int i=1; i<nx-1; ++i) {
        for(int j=1; j<ny-1; ++j) {
        T[i*ny+j]     = 0.0;
        T_new[i*ny+j] = 0.0;
        }
    }

    for(int i=0; i<nx; ++i){
        T[i*ny     ] = 100.0;
        T[i*ny+ny-1] = 0.0;
    }

    for(int j=0; j<ny; ++j){
        T[j]           = 0.0;
        T[(nx-1)*ny+j] = 0.0;
    }

#pragma acc data copy(T) create(T_new)    
{
    for (int it = 0; it<10000; ++it){
        erreur = 0.0;	
        #pragma acc parallel loop tile (32,32) reduction(max:erreur)
        for (int i=1; i<nx-1; ++i) {
            for (int j=1; j<ny-1; ++j) {
                idx = i*(ny)+j;
                T_new[idx] = 0.25*(T[idx+ny]+T[idx-ny] + T[idx+1]+T[idx-1]);
                erreur = fmax(erreur, fabs(T_new[idx]-T[idx]));
            }
        }
        if(it%100 == 0) fprintf(stderr,"it: %d, erreur: %e\n",it,erreur);

        #pragma acc parallel loop 
        for (int i=1; i<nx-1; ++i) {
            for (int j=1; j<ny-1; ++j) {
                T[i*(ny)+j] =  T_new[i*(ny)+j];
            }
        }
    }
}
    return 0;
}